In [ ]:
import ast
import pandas as pd
import plotly.express as px

BINANCE_FEE = 0.001 # 0.1% for Regular User https://www.binance.com/en/fee/trading

def _top_price(side_str: str) -> float | None:
    """Return the first price in a stringified list of bids/asks."""
    try:
        side = ast.literal_eval(side_str)
        return float(side[0][0]) if side else None
    except Exception:
        return None

def load_binance(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)           # columns: time, bids, asks
    df["best_bid"] = df["bids"].apply(_top_price)
    df["best_ask"] = df["asks"].apply(_top_price)

    return df[["time", "best_bid", "best_ask"]].sort_values("time")

def apply_fee(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds columns 'best_bid_with_fee' and 'best_ask_with_fee' to account for the Binance trading fee.
    """
    df["best_bid_with_fee"] = df["best_bid"] * (1 - BINANCE_FEE)  # Subtract fee for bids
    df["best_ask_with_fee"] = df["best_ask"] * (1 + BINANCE_FEE)  # Add fee for asks
    df["best_bid_with_fee_VIP9"] = df["best_bid"] * (1 - BINANCE_FEE)  # Subtract fee for bids
    df["best_ask_with_fee_VIP9"] = df["best_ask"] * (1 + BINANCE_FEE)  # Add fee for asks

    return df

# Unichain
df_unichain = pd.read_csv("/home/tobias/personal-dex-trading/out/data/latest_unichain_uniswap_v4_blocks.csv", index_col="timestamp")
df_unichain.index = pd.to_datetime(df_unichain.index, unit="s")  # seconds -> datetime
df_unichain.index += pd.Timedelta(hours=2)

# Binance
binance_df = load_binance("/home/tobias/personal-dex-trading/out/data/latest_binance_ws_orderbook.csv")
binance_df["time"] = pd.to_datetime(binance_df["time"])
binance_df = apply_fee(binance_df)

# merge df
merged_df =  pd.merge_asof(
    binance_df.sort_values("time"),
    df_unichain.sort_index(),
    left_on="time",
    right_index=True,
    direction="backward"
)
merged_df["bid_0.00001_weth"] = (merged_df["token1_outputs"].apply(ast.literal_eval).apply(lambda x: x[0][0] if isinstance(x, list) else None) / 10**6)
merged_df["ask_0.00001_weth"] = (merged_df["token1_inputs"].apply(ast.literal_eval).apply(lambda x: x[0][0] if isinstance(x, list) else None) / 10**6)
merged_df.head(2)

In [ ]:
import plotly.express as px

long_df = merged_df.melt(id_vars="time",
                         value_vars=["best_bid_with_fee", "best_ask_with_fee", "bid_0.00001_weth", "ask_0.00001_weth"],
                         var_name="Type",
                         value_name="Price")

# Plot with Plotly Express
fig = px.line(
    long_df,
    x="time",
    y="Price",
    color="Type",
    markers=True,
    title="Best Bids/Asks vs Bid/Ask for 0.00001 WETH",
    labels={"time": "Time", "Price": "Price", "Type": "Legend"}
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(showgrid=True, tickformat='%H:%M:%S'),
    yaxis=dict(showgrid=True),
    hovermode="x unified"  # Show hover labels for all lines
)

# Show the plot
fig.show()